In [ ]:
'''# Importing the libraries'''
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
'''#connect to the webpage'''
driver = webdriver.Chrome(executable_path=r'C:/Users/44814/Downloads/chromedriver_win32/chromedriver.exe')
driver.get("https://www.99acres.com/search/property/buy/residential-all/hinjewadipune?search_type=QS&refSection=GNB&search_location=CP12&lstAcn=CP_R&lstAcnId=12&src=CLUSTER&preference=S&selected_tab=1&city=19&res_com=R&property_type=R&isvoicesearch=N&keyword_suggest=Hinjewadi%2C%20Pune%3B&fullSelectedSuggestions=Hinjewadi%2C%20Pune&strEntityMap=W3sidHlwZSI6ImxvY2FsaXR5In0seyIxIjpbIkhpbmpld2FkaSwgUHVuZSIsIkNJVFlfMTksIExPQ0FMSVRZXzgzNiwgUFJFRkVSRU5DRV9TLCBSRVNDT01fUiJdfV0%3D&texttypedtillsuggestion=hin&refine_results=Y&Refine_Localities=Refine%20Localities&action=%2Fdo%2Fquicksearch%2Fsearch&suggestion=CITY_19%2C%20LOCALITY_836%2C%20PR%2C%20RESCOM_R&searchform=1&locality=836&price_min=null&price_max=null")
# Defining Dataframe to store fetch data
Property_data = pd.DataFrame()

# Pagenum starting from page 1
pagenum = 1
# Will fetch data of 20 pages 
count = 20
'''# loop initiate'''
while count !=0:
    my_page = driver.page_source
    #used wait to load the page completely 
    driver.implicitly_wait(30)
    WebDriverWait(driver,20)
    soup = BeautifulSoup(my_page, "html.parser") # get html
    scrap_data = soup.findAll('div', attrs={'class':'srpTuple__cardWrap'}) #contain all 25 card on the particular page
    
    for i in range(len(scrap_data)):
        #name of property
#         try:
#             Project = scrap_data[i].findAll('td', {'class':'srpTuple__propertyPremiumHeading srpTuple__spacer10 srpTuple__tdClasstwoPremium'})[0].text
#         except:
#             Project = ''
        try:
            Project_class = scrap_data[i].findAll('td', {'class':'srpTuple__propertyPremiumHeading srpTuple__spacer10 srpTuple__tdClasstwoPremium'})
            if len(Project_class) == 0:
                Project_class = scrap_data[i].findAll('td', {'class':'list_header_bold false srpTuple__tdClasstwoPremium srpTuple__tdClasstwoPremiumLandmark'})
                if len(Project_class) == 0:
                    Project_class = scrap_data[i].findAll('td', {'class':'cardContent__recomProjName list_header_bold spacer2'})
            Project = Project_class[0].text
        except IndexError:
            Project = ''
    
        features = scrap_data[i].findAll('td', {'class':'srpTuple__col title_semiBold'}) 
        try: 
    
            specification = features[2].text
    
        except:
            specification = ''
            
        try:
            area = features[1].text
    
        except:
            area = ''
            
        try:
            price = features[0].text
        except:
            price = ''
        
        print([Project,specification,area, price])
        
        Property_data = Property_data.append(pd.DataFrame([Project,specification,area, price]).T)
        
    count -= 1
        
    pagenum += 1
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        # New page via clicking on next
        LINK_TEXT = "https://www.99acres.com/property-in-hinjewadi-pune-ffid-page-" + str(pagenum)
        link_text = "Next Page"
        driver.find_element_by_partial_link_text("Next Page").click()
        print('Page:', pagenum)
        # time.sleep(2)
    except:
        break
# desired columns name        
Property_data.columns = ['Project','specification','area', 'price']
# stored the data in csv
Property_data.to_csv('property_data_.csv')      